<a href="https://www.kaggle.com/code/rafacasari/wokada-voice-changer" target="_parent"><img src="https://img.shields.io/badge/Open%20In%20Kaggle-035a7d?style=for-the-badge&logo=kaggle&logoColor=white" alt="Open In Colab"/></a>

### [w-okada's Voice Changer](https://github.com/w-okada/voice-changer) | **Kaggle**

---

## **⬇ VERY IMPORTANT ⬇**

You can use the following settings for better results:

If you're using a index: `f0: RMVPE_ONNX | Chunk: 112 or higher | Extra: 8192`<br>
If you're not using a index: `f0: RMVPE_ONNX | Chunk: 96 or higher | Extra: 16384`<br>
**Don't forget to select a GPU in the GPU field, <b>NEVER</b> use CPU!
> Seems that PTH models performance better than ONNX for now, you can still try ONNX models and see if it satisfies you


*You can always [click here](https://github.com/YunaOneeChan/Voice-Changer-Settings) to check if these settings are up-to-date*

---
**Credits**<br>
Realtime Voice Changer by [w-okada](https://github.com/w-okada)<br>
Notebook files updated by [rafacasari](https://github.com/Rafacasari)<br>
Recommended settings by [YunaOneeChan](https://github.com/YunaOneeChan)

**Need help?** [AI Hub Discord](https://discord.gg/aihub) » ***#help-realtime-vc***

---

# Kaggle Tutorial
Running this notebook can be a bit complicated.\
After created your Kaggle account, you'll need to **verify your phone number** to be able to use Internet Connection and GPUs.\
Follow the instructions on the image below.

## <font color=blue>*You can use <b>GPU P100</b> instead of GPU T4, some people are telling that <b>P100 is better</b>.*</font>
![instructions.png](https://i.imgur.com/0NutkD8.png)

# Clone repository and install dependencies
This first step will download the latest version of Voice Changer and install the dependencies. **It will take some time to complete.**

In [0]:
# This will make that we're on the right folder before installing
%cd /kaggle/working/

!pip install colorama --quiet
from colorama import Fore, Style
import os

print(f"{Fore.CYAN}> Cloning the repository...{Style.RESET_ALL}")
!git clone https://github.com/w-okada/voice-changer.git --quiet --depth 1
print(f"{Fore.GREEN}> Successfully cloned the repository!{Style.RESET_ALL}")
%cd voice-changer/server/

print(f"{Fore.CYAN}> Installing libportaudio2...{Style.RESET_ALL}")
!apt-get -y install libportaudio2 -qq

print(f"{Fore.CYAN}> Installing pre-dependencies...{Style.RESET_ALL}")
# Install dependencies that are missing from requirements.txt and pyngrok
!pip install faiss-gpu fairseq pyngrok --quiet 
!pip install pyworld --no-build-isolation --quiet
print(f"{Fore.CYAN}> Installing dependencies from requirements.txt...{Style.RESET_ALL}")
!pip install -r requirements.txt --quiet

# Download the default settings ^-^
if not os.path.exists("/kaggle/working/voice-changer/server/stored_setting.json"):
    !wget -q https://gist.githubusercontent.com/Rafacasari/d820d945497a01112e1a9ba331cbad4f/raw/8e0a426c22688b05dd9c541648bceab27e422dd6/kaggle_setting.json -O /kaggle/working/voice-changer/server/stored_setting.json
print(f"{Fore.GREEN}> Successfully installed all packages!{Style.RESET_ALL}")

print(f"{Fore.GREEN}> You can safely ignore the dependency conflict errors, it's a error from Kaggle and don't interfer on Voice Changer!{Style.RESET_ALL}")

# Start Server **using ngrok**
This cell will start the server, the first time that you run it will download the models, so it can take a while (~1-2 minutes)

---
You'll need a ngrok account, but <font color=green>**it's free**</font> and easy to create!
---
**1** - Create a **free** account at [ngrok](https://dashboard.ngrok.com/signup)\
**2** - If you didn't logged in with Google or Github, you will need to **verify your e-mail**!\
**3** - Click [this link](https://dashboard.ngrok.com/get-started/your-authtoken) to get your auth token, and replace **YOUR_TOKEN_HERE** with your token.\
**4** - *(optional)* Change to a region near to you

In [ ]:
# ---------------------------------
# SETTINGS
# ---------------------------------

Token = '2Tn2hbfLtw2ii6DHEJy7SsM1BjI_21G14MXSwz7qZSDL2Dv3B'
ClearConsole = True # Clear console after initialization. Set to False if you are having some error, then you will be able to report it.
Region = "sa" # Read the instructions below

# You can change the region for a better latency, use only the abbreviation
# Choose between this options: 
# us -> United States (Ohio)
# ap -> Asia/Pacific (Singapore)
# au -> Australia (Sydney)
# eu -> Europe (Frankfurt)
# in -> India (Mumbai)
# jp -> Japan (Tokyo)
# sa -> South America (Sao Paulo)

# ---------------------------------
# DO NOT TOUCH ANYTHING DOWN BELOW!
# ---------------------------------

%cd /kaggle/working/voice-changer/server
    
from pyngrok import conf, ngrok
MyConfig = conf.PyngrokConfig()
MyConfig.auth_token = Token
MyConfig.region = Region
#conf.get_default().authtoken = Token
#conf.get_default().region = Region
conf.set_default(MyConfig);

import subprocess, threading, time, socket, urllib.request
PORT = 8000

from pyngrok import ngrok
ngrokConnection = ngrok.connect(PORT)
public_url = ngrokConnection.public_url

from IPython.display import clear_output

def wait_for_server():
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', PORT))
        if result == 0:
            break
        sock.close()
    if ClearConsole:
        clear_output()
    print("--------- SERVER READY! ---------")
    print("Your server is available at:")
    print(public_url)
    print("---------------------------------")

threading.Thread(target=wait_for_server, daemon=True).start()

!python3 MMVCServerSIO.py \
  -p {PORT} \
  --https False \
  --content_vec_500 pretrain/checkpoint_best_legacy_500.pt \
  --content_vec_500_onnx pretrain/content_vec_500.onnx \
  --content_vec_500_onnx_on true \
  --hubert_base pretrain/hubert_base.pt \
  --hubert_base_jp pretrain/rinna_hubert_base_jp.pt \
  --hubert_soft pretrain/hubert/hubert-soft-0d54a1f4.pt \
  --nsf_hifigan pretrain/nsf_hifigan/model \
  --crepe_onnx_full pretrain/crepe_onnx_full.onnx \
  --crepe_onnx_tiny pretrain/crepe_onnx_tiny.onnx \
  --rmvpe pretrain/rmvpe.pt \
  --model_dir model_dir \
  --samples samples.json

ngrok.disconnect(ngrokConnection.public_url)